<a href="https://colab.research.google.com/github/Yin-Tzu/predict_DA_reaction_product/blob/main/load_model_4th%E6%B2%92%E6%94%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

把檔案放進目錄

In [2]:
import zipfile
file_dir = './0.zip'
zipFile = zipfile.ZipFile(file_dir)
for file in zipFile.namelist():
    zipFile.extract(file, '/content')  # 解压路径
zipFile.close()

使用RDkit須執行的程式

In [1]:
%%bash
wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
conda config --set always_yes yes --set changeps1 no
conda install -q -y -c conda-forge python=3.7
conda install -q -y -c conda-forge rdkit==2020.09.2



PREFIX=/usr/local
Unpacking payload ...
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - brotlipy==0.7.0=py38h27cfd23_1003
    - ca-certificates==2020.10.14=0
    - certifi==2020.6.20=pyhd3eb1b0_3
    - cffi==1.14.3=py38h261ae71_2
    - chardet==3.0.4=py38h06a4308_1003
    - conda-package-handling==1.7.2=py38h03888b9_0
    - conda==4.9.2=py38h06a4308_0
    - cryptography==3.2.1=py38h3c74f83_1
    - idna==2.10=py_0
    - ld_impl_linux-64==2.33.1=h53a641e_7
    - libedit==3.1.20191231=h14c3975_1
    - libffi==3.3=he6710b0_2
    - libgcc-ng==9.1.0=hdf63c60_0
    - libstdcxx-ng==9.1.0=hdf63c60_0
    - ncurses==6.2=he6710b0_1
    - openssl==1.1.1h=h7b6447c_0
    - pip==20.2.4=py38h06a4308_0
    - pycosat==0.6.3=py38h7b6447c_1
    - pycparser==2.20=py_2
    - pyopenssl==19.1.0=pyhd3eb1b0_1
    - pysocks==1.7.1=py38h06a4308_0
    - python==3.8.5=h7579374_1
    - readline==8.0=h7b6447c_

--2021-01-25 10:47:55--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2021-01-25 10:47:55--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94235922 (90M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 12.9M 7s
    50K .......... .......... .......... .......... ..........

使用RDkit須執行的程式

In [3]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

try:
  from rdkit import Chem
  from rdkit.Chem.Draw import IPythonConsole
except ImportError:
  print('Stopping RUNTIME. Colaboratory will restart automatically. Please run cell again.')
  exit()

3D圖

In [4]:
!pip install py3Dmol

In [8]:
import cv2
import copy
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import joblib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import random
import copy
import time


tStart = time.time()#計時開始
#MiMaScaler = joblib.load('scalar02')  # 有數值縮放才需要，這次沒有

# 算階層
def x(num):  # https://www.runoob.com/python3/python3-factorial.html
  factorial = 1
  for i in range(1,num + 1):
    factorial = factorial*i
  return factorial

# 
def HOMOLUMO(load_plt):
  # 步驟一前處理
  im = cv2.imread(load_plt)
  dst = cv2.resize(im, (128,128), interpolation=cv2.INTER_AREA)
  dst = copy.deepcopy(dst[np.newaxis, :])
  # 步驟一
  model = tf.keras.models.load_model('my_model2.h5')
  prediction = model.predict_classes(dst)
  print(prediction[0])
  # 步驟二
  if prediction[0]==0:
      b = 2
  if prediction[0] == 1:
      b = 4
  if prediction[0] == 2:
      b = 6
  if prediction[0] == 3:
      b = 8
  if prediction[0] == 4:
      b = 10
  if prediction[0] == 5:
      b = 12
  if prediction[0] == 6:
      b = 14
  if prediction[0] == 7:
      b = 16
  if prediction[0] == 8:
      b = 18
  if prediction[0] == 9:
      b = 20
  H=np.zeros([b, b]).astype('int64')
  for j in range(b-1):
      p1 = j + 1
      p2 = j
      H[p1, p2] = 1
      H[p2, p1] = 1
  #print(H) 
  # 步驟三
  eigvals, eigvecs = la.eig(H)
  #print(eigvals)
  #print(eigvecs)
  # 步驟四前處理
  img=[]
  c = int((20-b)/2)
  for e in range(b):
    b1 = eigvecs[:, e].reshape(b, 1)
    #print(b1,'b1')
    bbb = copy.deepcopy(b1)
    for d in range(c):  # 插入0
      bbb = np.insert(bbb, np.size(bbb,0), np.zeros(1), 0)
      bbb = np.insert(bbb, 0, np.zeros(1), 0)
    bbb = copy.deepcopy(bbb[:,:, np.newaxis])
    bbb = copy.deepcopy(bbb[np.newaxis, :])
    if len(img) == 0:
      img = copy.deepcopy(bbb)
    else:
      img = np.concatenate((img, bbb), axis=0)
  p_img=[]
  for i in range(len(img)):
    true_img = copy.deepcopy(img[i])
    r1= random.randint(0,(len(img)-1))
    r2= random.randint(0,(len(img)-1))
    concat_img = copy.deepcopy(np.concatenate((true_img, img[r1]), axis=1))
    concat_img = np.concatenate((img[r2], concat_img), axis=1)
    concat_img = copy.deepcopy(concat_img[np.newaxis, :])
    if len(p_img) == 0:
      p_img = copy.deepcopy(concat_img)
    else:
      p_img = np.concatenate((p_img, concat_img), axis=0)
  #print(p_img.shape)
  # 步驟四
  model = tf.keras.models.load_model('eig_model.h5')
  for i in range(len(p_img)):
    ss = p_img[i]
    ss2 = copy.deepcopy(ss[np.newaxis, :])
    prediction = model.predict_classes(ss2)
    print(prediction[0])
    if prediction[0]==0:
      HOMO=ss[:,1,:]
      #print(HOMO,'0')
    if prediction[0]==1:
      LUMO=ss[:,1,:]
      #print(LUMO.shape,'0')
  return HOMO, LUMO,b




load = u'./14.png'
load2 = u'./15_4.png'
HOMO1, LUMO1,b1=HOMOLUMO(load)
HOMO2, LUMO2,b2=HOMOLUMO(load2)

#步驟5前處理
r=[HOMO1, LUMO1, HOMO2, LUMO2]
#rgbMO=[]
def layer3(MO, b12, revers=False):
  rgbMO=[]
  for rev in range(2):
    if (rev==1) and (revers==False):
      break
    pn =int(x(b12)/((x(b12-2))*(x(2))))  # 每個軌域中選出兩個不同碳，有幾種分法
    ppnn = 0
    n=int(10-(b12/2))
    #print(n,'n')
    #print(al,pn)
    c = int((20-b12)/2)
    rr, r1, r2 = [], 0, 0
    while ppnn != pn:  # 算次數，次數沒到繼續執行
      while ([r1,r2] in rr) or ([r2,r1] in rr) or (r1==r2): # 隨機產生r
        r1 = random.randint(0,b12-1)
        r2 = random.randint(0,b12-1)
      rr.append([r1,r2])
      if rev==0:  # LUMO要轉才會讓不同方向配對到
        al=copy.deepcopy(MO[n:b12+n][:,:,np.newaxis])  #list(range((20)))
        #print(al)
        if (len(rr)<2) and (revers==True):
          print(al,'al')
      else:
        al=copy.deepcopy(MO[n:b12+n][::-1][:,:,np.newaxis])
        if len(rr)<2:
          print(al,'alr')
      g=np.zeros(b12).reshape(b12,1,1)  # 第二層:綠色
      bl = np.zeros(b12).reshape(b12,1,1)  # 第三層:藍色
      g[r1] = copy.deepcopy(al[r1])  # 紅色與綠色換數值
      g[r2] = copy.deepcopy(al[r2])  # 紅色與綠色換數值
      al[r1] = copy.deepcopy(0)  # 紅色與綠色換數值
      al[r2] = copy.deepcopy(0)  # 紅色與綠色換數值
      #print(al,g)
      gg = []
      for i in [al,g,bl]:  # 紅綠藍都變成20*1
        for d in range(c):
          i = np.insert(i, 0, np.zeros(1), 0)
          i = np.insert(i, np.size(i,0), np.zeros(1), 0)
        gg.append(i)
      al = copy.deepcopy(gg[0])
      g = copy.deepcopy(gg[1])
      bl = copy.deepcopy(gg[2])
      reg = np.append(al, g, axis=2)  #　（：：在這裡合併）
      regbl = np.append(reg, bl, axis=2)
      regbl = copy.deepcopy(regbl[np.newaxis, :])
      if rgbMO==[]:
        rgbMO = copy.deepcopy(regbl)
      else:
        #print(rgbMO.shape)
        rgbMO = copy.deepcopy(np.concatenate((rgbMO, regbl), axis=0))
      #print(rgbMO.shape)
      ppnn+=1
  return rgbMO
HOMO1=layer3(HOMO1,b1)
LUMO1=layer3(LUMO1,b1, revers=True)
HOMO2=layer3(HOMO2,b2)
LUMO2=layer3(LUMO2,b2, revers=True)
print(HOMO1.shape, LUMO1.shape)
print(HOMO2.shape, LUMO2.shape)
b23 = []
for i in range(len(HOMO1)):
  for j in range(len(LUMO2)):
    reaction_rgb = np.append(HOMO1[i], LUMO2[j], axis=1)  # HOMOLUMO合併
    b23.append([b1,b2])
    reaction_rgb = copy.deepcopy(reaction_rgb[np.newaxis, :])
    if i==0 and j==0:
      reaction_rgb12 = copy.deepcopy(reaction_rgb)
    else:
      reaction_rgb12 = copy.deepcopy(np.concatenate([reaction_rgb12, reaction_rgb]))
for i in range(len(HOMO2)):
  for j in range(len(LUMO1)):
    reaction_rgb = copy.deepcopy(np.append(HOMO2[i], LUMO1[j], axis=1))  # HOMOLUMO合併
    b23.append([b2,b1])
    #if i==1 and j==1:
    #  print(reaction_rgb[:,0,0])
    reaction_rgb = copy.deepcopy(reaction_rgb[np.newaxis, :])
    reaction_rgb12 = copy.deepcopy(np.concatenate([reaction_rgb12, reaction_rgb]))
#print(reaction_rgb12.shape, len(b23))

# 步驟5
HOMOSubstituent, LUMOSubstituent=[],[]
model = tf.keras.models.load_model('NN_3_2.h5')
for i in range(len(reaction_rgb12)):
  p=reaction_rgb12[i][np.newaxis, :]
  #print(p.shape)
  prediction = model.predict_classes(p)
  #步驟六前處理
  if prediction[0]==1:
    HOMOc = int((20-b23[i][0])/2)  # b23[i][0]是碳數，有可能是b2 or b1
    HOMOlacktwonumber = reaction_rgb12[i][:,0,0][HOMOc:HOMOc+b23[i][0]]
    oneHOMOSubstituent=copy.deepcopy([])
    n=0
    for j in range(len(HOMOlacktwonumber)):  # HOMO中的各個數值的索引號
      
      if HOMOlacktwonumber[j]==0:
        #print(j, len(HOMOlacktwonumber))
        if n==0:
          oneHOMOSubstituent.append((j))
        else:
          oneHOMOSubstituent.append((j-oneHOMOSubstituent[0]+1))
          oneHOMOSubstituent.append((len(HOMOlacktwonumber)-j-1))
        n+=1
    HOMOSubstituent.append(oneHOMOSubstituent)
    
    LUMOc = int((20-b23[i][1])/2)  # b23[i][0]是碳數，有可能是b2 or b1
    LUMOlacktwonumber = reaction_rgb12[i][:,1,0][LUMOc:LUMOc+b23[i][1]]
    #print(reaction_rgb12[i][:,1,0][LUMOc:LUMOc+b23[i][1]])
    oneLUMOSubstituent=copy.deepcopy([])
    n=0
    for j in range(len(LUMOlacktwonumber)):
      if LUMOlacktwonumber[j]==0:
        if n==0:
          oneLUMOSubstituent.append((j))
        else:
          oneLUMOSubstituent.append((j-oneLUMOSubstituent[0]+1))
          oneLUMOSubstituent.append((len(LUMOlacktwonumber)-j-1))
        n+=1
    LUMOSubstituent.append(oneLUMOSubstituent)
print(LUMOSubstituent)  # 最後步驟要用到
print(HOMOSubstituent)  # 最後步驟要用到

ringssmiles, Substituentssmiles, repe,HOMOSubstituentnotrepe,LUMOSubstituentnotrepe=[],[],[],[],[]
for i in range(len(LUMOSubstituent)):
  if [HOMOSubstituent[i],LUMOSubstituent[i]] not in repe:
    repe.append([HOMOSubstituent[i],LUMOSubstituent[i]])
    HOMOSubstituentnotrepe.append(HOMOSubstituent[i])
    LUMOSubstituentnotrepe.append(LUMOSubstituent[i])
  else:
    continue
  ring = copy.deepcopy(['C1=','C1'])
  ringCnumber = HOMOSubstituent[i][1]+LUMOSubstituent[i][1]-2
  ring.insert(1,'C'*ringCnumber)
  ringssmiles.append("".join(ring))
  amoleculeSubstituents=copy.deepcopy([HOMOSubstituent[i][0],LUMOSubstituent[i][0],LUMOSubstituent[i][2],HOMOSubstituent[i][2]])
  amoleculeSubstituentssmiles=copy.deepcopy([])                                          
  for j in amoleculeSubstituents:
    if j==0:
      amoleculeSubstituentssmiles.append("")
    else:
      oneSubstituentsmiles=copy.deepcopy([])
      for ij in range(j):
        if ij%2==0:
          oneSubstituentsmiles.append("C")
        else:
          oneSubstituentsmiles.append('=C')
      amoleculeSubstituentssmiles.append("".join(oneSubstituentsmiles))
  Substituentssmiles.append(amoleculeSubstituentssmiles)
print(Substituentssmiles)  # 最後步驟要用到
print(ringssmiles)  # 最後步驟要用到


# 步驟六
from store_substituent2 import differentmolecule
for i in range(len(ringssmiles)):
  print(HOMOSubstituentnotrepe[i],LUMOSubstituentnotrepe[i])
  differentmolecule(ringssmiles[i],Substituentssmiles[i][0],Substituentssmiles[i][1],Substituentssmiles[i][2],Substituentssmiles[i][3],HOMOSubstituentnotrepe[i][1],LUMOSubstituentnotrepe[i][1])


tEnd = time.time()#計時結束
#列印結果
print("It cost %f sec" % (tEnd - tStart))  #會自動做近位
print(tEnd - tStart)  #原型長這樣

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


6
6
6
6
6
5
3
1
0
2
6
6
6
6
4
1
3
1
2
0
[[[ 0.36314805]]

 [[-0.07591862]]

 [[-0.34727674]]

 [[ 0.14851922]]

 [[ 0.31622777]]

 [[-0.21462883]]

 [[-0.27135812]]

 [[ 0.27135812]]

 [[ 0.21462883]]

 [[-0.31622777]]

 [[-0.14851922]]

 [[ 0.34727674]]

 [[ 0.07591862]]

 [[-0.36314805]]] al
[[[-0.36314805]]

 [[ 0.07591862]]

 [[ 0.34727674]]

 [[-0.14851922]]

 [[-0.31622777]]

 [[ 0.21462883]]

 [[ 0.27135812]]

 [[-0.27135812]]

 [[-0.21462883]]

 [[ 0.31622777]]

 [[ 0.14851922]]

 [[-0.34727674]]

 [[-0.07591862]]

 [[ 0.36314805]]] alr
[[[ 0.60150096]]

 [[-0.37174803]]

 [[-0.37174803]]

 [[ 0.60150096]]] al
[[[ 0.60150096]]

 [[-0.37174803]]

 [[-0.37174803]]

 [[ 0.60150096]]] alr
(91, 20, 1, 3) (182, 20, 1, 3)
(6, 20, 1, 3) (12, 20, 1, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:158: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


[[0, 2, 2], [0, 2, 2], [0, 2, 2], [0, 2, 2], [0, 2, 2], [0, 2, 2], [2, 2, 0], [2, 2, 0], [2, 2, 0], [2, 2, 0], [2, 2, 0], [2, 2, 0], [10, 2, 2], [2, 2, 10], [8, 2, 4], [0, 2, 12], [12, 2, 0], [4, 2, 8]]
[[8, 4, 2], [8, 4, 2], [0, 4, 10], [0, 4, 10], [4, 4, 6], [4, 4, 6], [10, 4, 0], [10, 4, 0], [6, 4, 4], [6, 4, 4], [2, 4, 8], [2, 4, 8], [0, 4, 0], [0, 4, 0], [0, 4, 0], [0, 4, 0], [0, 4, 0], [0, 4, 0]]
[['C=CC=CC=CC=C', '', 'C=C', 'C=C'], ['', '', 'C=C', 'C=CC=CC=CC=CC=C'], ['C=CC=C', '', 'C=C', 'C=CC=CC=C'], ['C=CC=CC=CC=CC=C', 'C=C', '', ''], ['C=CC=CC=C', 'C=C', '', 'C=CC=C'], ['C=C', 'C=C', '', 'C=CC=CC=CC=C'], ['', 'C=CC=CC=CC=CC=C', 'C=C', ''], ['', 'C=C', 'C=CC=CC=CC=CC=C', ''], ['', 'C=CC=CC=CC=C', 'C=CC=C', ''], ['', '', 'C=CC=CC=CC=CC=CC=C', ''], ['', 'C=CC=CC=CC=CC=CC=C', '', ''], ['', 'C=CC=C', 'C=CC=CC=CC=C', '']]
['C1=CCCCC1', 'C1=CCCCC1', 'C1=CCCCC1', 'C1=CCCCC1', 'C1=CCCCC1', 'C1=CCCCC1', 'C1=CCCCC1', 'C1=CCCCC1', 'C1=CCCCC1', 'C1=CCCCC1', 'C1=CCCCC1', 'C1=CCCCC1']
[8, 

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

...

None

[0, 4, 0] [4, 2, 8]
6 CHI_TETRAHEDRAL_CW
6 CHI_TETRAHEDRAL_CW


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

None

It cost 93.294888 sec
93.29488778114319
